In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog
from tkinter import Label
from PIL import ImageTk


In [2]:
normal_path = "Dataset/Normal/"
stroke_path = "Dataset/Stroke/"
normal_folder = os.listdir(normal_path)
stroke_folder = os.listdir(stroke_path)

print("Images in Normal Data:", len(normal_folder))
print("Images in Stroke Data:", len(stroke_folder))

data = []

for img_file in normal_folder:
    image = Image.open(normal_path + img_file)
    image = image.resize((224, 224))
    image = image.convert('RGB')
    data.append(np.array(image))
    
for img_file in stroke_folder:
    image = Image.open(stroke_path + img_file)
    image = image.resize((224, 224))
    image = image.convert('RGB')
    data.append(np.array(image))

normal_label = [0] * len(normal_folder)
stroke_label = [1] * len(stroke_folder)
Target_label = normal_label + stroke_label

x = np.array(data)
y = np.array(Target_label)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, shuffle=True)

x_train_s = x_train / 255.0
x_test_s = x_test / 255.0


Images in Normal Data: 1551
Images in Stroke Data: 950


In [3]:
model = Sequential()

model.add(Conv2D(100, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(80, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(500, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 100)       │           2,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 100)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 80)        │          72,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 80)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 64)          │          46,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 43264)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 500)                 │      21,632,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             501 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,004,525 (83.94 MB)

 Trainable params: 22,004,525 (83.94 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = model.fit(x_train_s, y_train, batch_size=32, epochs=10, validation_data=(x_test_s, y_test))

loss, acc = model.evaluate(x_test_s, y_test)
print("Loss on Test Data:", loss)
print("Accuracy on Test Data:", acc)

loss, acc = model.evaluate(x_train_s, y_train)
print("Loss on Train Data:", loss)
print("Accuracy on Train Data:", acc)


Epoch 1/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 310s 4s/step - accuracy: 0.6147 - loss: 0.6955 - val_accuracy: 0.7769 - val_loss: 0.4856
Epoch 2/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 300s 4s/step - accuracy: 0.8438 - loss: 0.3655 - val_accuracy: 0.9243 - val_loss: 0.1515
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 307s 4s/step - accuracy: 0.9588 - loss: 0.1143 - val_accuracy: 0.9602 - val_loss: 0.0935
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 301s 4s/step - accuracy: 0.9726 - loss: 0.0761 - val_accuracy: 0.9482 - val_loss: 0.1511
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 306s 4s/step - accuracy: 0.9824 - loss: 0.0483 - val_accuracy: 0.9761 - val_loss: 0.0687
Epoch 6/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 312s 4s/step - accuracy: 0.9927 - loss: 0.0181 - val_accuracy: 0.9841 - val_loss: 0.0482
Epoch 7/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 301s 4s/step - accuracy: 0.9965 - loss: 0.0091 - val_accuracy: 0.9761 - val_loss: 0.0913
Epoch 8/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 301s 4s/step - accuracy: 0.9963 - loss: 0.0113 - val_accuracy: 0.9801 - v

In [5]:
model.save("stroke_detection_model.h5")

In [1]:
import tkinter as tk
from tkinter import filedialog, Label, messagebox, Button
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf 

class StrokeDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Brain stroke detection")
        self.root.geometry("800x600")
        self.root.configure(bg="#E9F3FB")  

        custom_font = ("Helvetica", 18, "bold")
        
        label = tk.Label(root, text="Brain Stroke detection using machine learning", bg='#E9F3FB', font=custom_font, fg='#0B2F9F')
        label.pack(pady=20)

        self.panel_frame = tk.Frame(self.root, bg="#FFFFFF", bd=2, relief="solid", highlightbackground="#0B2F9F", highlightthickness=2)
        self.panel_frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER, width=600, height=400)

        self.inner_frame = tk.Frame(self.panel_frame, bg="#FFFFFF", relief="flat", bd=0)
        self.inner_frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER, width=596, height=396)

        self.inner_frame.config(highlightbackground="#0B2F9F", highlightcolor="#0B2F9F", highlightthickness=2)

        self.load_placeholder_image()

        self.label = tk.Label(self.inner_frame, text="Drag & Drop\nor browse", font=("Arial", 16), fg="#295F98", bg="#FFFFFF", justify=tk.CENTER)
        self.label.place(relx=0.5, rely=0.65, anchor=tk.CENTER)

        self.large_image_label = tk.Label(self.root, bg="#E9F3FB")
        self.large_image_label.place(relx=0.5, rely=0.3, anchor=tk.CENTER, width=500, height=300)
        self.large_image_label.place_forget()

        self.support_text = tk.Label(self.inner_frame, text="Supports: JPEG, JPG, PNG", font=("Arial", 10), fg="#A0A0A0", bg="#FFFFFF")
        self.support_text.place(relx=0.5, rely=0.85, anchor=tk.CENTER)

        self.image_reference = None

        self.result_label = tk.Label(self.root, text="", font=("Arial", 16), fg="green", bg='#FFFFFF')
        self.result_label.place(relx=0.5, rely=0.8, anchor=tk.CENTER)

        self.retry_button = tk.Button(self.root, text="Retry", command=self.reset, font=("Arial", 14), bg="#295F98", fg="#FFFFFF", padx=20)
        self.retry_button.place(relx=0.5, rely=0.9, anchor=tk.CENTER)
        self.retry_button.place_forget()  

        self.inner_frame.bind("<Button-1>", lambda e: self.load_image())

    def load_placeholder_image(self):
        try:
            self.icon_img = Image.open("placeholder.jpg").resize((150, 150)) 
            self.icon_img_tk = ImageTk.PhotoImage(self.icon_img)
            
            self.icon_label = tk.Label(self.inner_frame, image=self.icon_img_tk, bg="#FFFFFF")
            self.icon_label.image = self.icon_img_tk  
            self.icon_label.place(relx=0.5, rely=0.3, anchor=tk.CENTER)

            self.image_reference = self.icon_img_tk

        except Exception as e:
            messagebox.showerror("Error", f"Failed to load image: {e}")

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            try:
                selected_image = Image.open(file_path).resize((500, 300))  
                selected_image_tk = ImageTk.PhotoImage(selected_image)

                self.large_image_label.place(x=150, y=120)  
                self.large_image_label.config(image=selected_image_tk)
                self.large_image_label.image = selected_image_tk 
                
                self.icon_label.place_forget()
                self.label.place_forget()

                prediction_result = self.predict_stroke(file_path)
                self.result_label.config(text=f"Prediction: {prediction_result}")

                self.retry_button.config(state=tk.NORMAL)
                self.retry_button.place(relx=0.5, rely=0.9, anchor=tk.CENTER)

            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {e}")

    def predict_stroke(self, image_path):
        try:
            image = Image.open(image_path).resize((224, 224))  
            image = image.convert('RGB')  
            image_array = np.array(image) / 255.0  
            image_array = np.expand_dims(image_array, axis=0)  
            
            model = tf.keras.models.load_model('stroke_detection_model.h5')
            
            prediction = model.predict(image_array)

            if prediction[0] > 0.5:
                return "Stroke"
            else:
                return "No Stroke"
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {e}")
            return "Error"

    def reset(self):
        self.label.place(relx=0.5, rely=0.65, anchor=tk.CENTER)
        self.icon_label.place(relx=0.5, rely=0.3, anchor=tk.CENTER)
        self.large_image_label.config(image='')
        self.result_label.config(text="")
        self.retry_button.place_forget() 
        self.image_uploaded = False 

root = tk.Tk()
app = StrokeDetectionApp(root)
root.mainloop()


In [7]:
model = tf.keras.models.load_model("stroke_detection_model.h5")